In [2]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 664.7 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=05653139dc5efc5770cb62156cb4bfaa283e433a370cad22af09e54a630f053e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# let's now print how the Dataset looks like
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})

In [5]:
raw_datasets.column_names

{'train': ['tokens', 'ner_tags'],
 'test': ['tokens', 'ner_tags'],
 'validation': ['tokens', 'ner_tags']}

In [6]:
# let's print an instance of dataset
idx=1000
rec=raw_datasets['train'][idx]
for w, t in zip(rec['tokens'],rec['ner_tags']):
  print('{}\t{}'.format(w,t))

सूक्ष्म	0
,	0
लघु	0
एवं	0
मध्यम	0
उद्यम	0
(	0
एमएसएमई	3
)	0
और	0
सड़क	3
परिवहन	4
एवं	4
राजमार्ग	4
मंत्री	0
श्री	0
नितिन	1
गडकरी	2
ने	0
आज	0
जानकारी	0
दी	0
कि	0
उनका	0
मंत्रालय	0
एक	0
कृषि	0
एमएसएमई	0
नीति	0
लाने	0
पर	0
काम	0
कर	0
रहा	0
है	0
जो	0
स्थानीय	0
कच्चे	0
माल	0
का	0
उपयोग	0
करते	0
हुए	0
विनिर्माण	0
उत्पादों	0
के	0
लिए	0
ग्रामीण	0
,	0
जनजातीय	0
,	0
कृषि	0
और	0
वन	0
क्षेत्रों	0
में	0
उद्यमिता	0
विकास	0
पर	0
ध्यान	0
केंद्रित	0
करेगी	0
।	0


In [7]:
column_names = raw_datasets["train"].column_names
print(column_names)

features = raw_datasets["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [8]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [9]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)

{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [10]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indicNER', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indicNER")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indicNER', num_labels=num_labels )

2024-03-13 08:48:52.321020: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 08:48:52.321115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 08:48:52.498416: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

model=model.to("cuda")

In [12]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [13]:
ot=len(raw_datasets["train"])
nts=int(ot*0.2)
train_dataset = raw_datasets["train"].select(range(20000))
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/5 [00:00<?, ?ba/s]

In [14]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [15]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [17]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)

args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    learning_rate=1e-5
)

In [18]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

In [19]:
trainer.args

TrainingArguments(
_n_gpu=2,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_

In [20]:
train_result = trainer.train()
metrics = train_result.metrics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.686400,0.182919,0.802999,0.844120,0.823046,10213,0.671322,0.687717,0.679421,9786,0.795491,0.838096,0.816238,10568,0.758990,0.791965,0.775127,0.947088
2,0.153000,0.177808,0.815602,0.852737,0.833756,10213,0.671673,0.699264,0.685191,9786,0.797934,0.840367,0.818601,10568,0.763627,0.799326,0.781069,0.948005
3,0.135700,0.179740,0.814580,0.855576,0.834575,10213,0.676801,0.696096,0.686313,9786,0.801449,0.837623,0.819137,10568,0.766467,0.798312,0.782065,0.948051


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [21]:
model.save_pretrained('my_indicNER')

In [22]:
tokenizer.save_pretrained("tokenizer_indicNER")

('tokenizer_indicNER/tokenizer_config.json',
 'tokenizer_indicNER/special_tokens_map.json',
 'tokenizer_indicNER/vocab.txt',
 'tokenizer_indicNER/added_tokens.json',
 'tokenizer_indicNER/tokenizer.json')

In [23]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [24]:
import json

In [25]:
config = json.load(open("/kaggle/working/my_indicNER/config.json"))

In [26]:
config["id2label"] = id2label
config["label2id"] = label2id

In [27]:
json.dump(config, open("/kaggle/working/my_indicNER/config.json","w"))

In [28]:
model = AutoModelForTokenClassification.from_pretrained('/kaggle/working/my_indicNER')

In [29]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [30]:
!pip install torch
import torch

In [31]:
def get_ner(sentence):
    tok_sentence = tokenizer(sentence, return_tensors='pt')

    with torch.no_grad():
        logits = model(**tok_sentence).logits.argmax(-1)
        predicted_tokens_classes = [
            model.config.id2label[t.item()] for t in logits[0]]

        predicted_labels = []

        previous_token_id = 0
        word_ids = tok_sentence.word_ids()
        for word_index in range(len(word_ids)):
            if word_ids[word_index] == None:
                previous_token_id = word_ids[word_index]
            elif word_ids[word_index] == previous_token_id:
                previous_token_id = word_ids[word_index]
            else:
                predicted_labels.append(predicted_tokens_classes[word_index])
                previous_token_id = word_ids[word_index]

        ner_output = []
        for index in range(len(sentence.split(' '))):
            if(index<len(predicted_labels)):
                ner_output.append((sentence.split(' ')[index], predicted_labels[index]))
            else:
                ner_output.append((sentence.split(' ')[index], 'O'))
        return ner_output

In [33]:
!ls

my_indicNER  output_dir  tokenizer_indicNER  wandb


In [34]:
!zip -r BertTuned.zip /kaggle/working/my_indicNER

  adding: kaggle/working/my_indicNER/ (stored 0%)
  adding: kaggle/working/my_indicNER/config.json (deflated 50%)
  adding: kaggle/working/my_indicNER/model.safetensors (deflated 7%)


In [35]:
!zip -r BertTuned_tokenizer.zip /kaggle/working/tokenizer_indicNER

  adding: kaggle/working/tokenizer_indicNER/ (stored 0%)
  adding: kaggle/working/tokenizer_indicNER/tokenizer.json (deflated 69%)
  adding: kaggle/working/tokenizer_indicNER/special_tokens_map.json (deflated 80%)
  adding: kaggle/working/tokenizer_indicNER/vocab.txt (deflated 48%)
  adding: kaggle/working/tokenizer_indicNER/tokenizer_config.json (deflated 75%)


In [36]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

  0%|          | 0/3 [00:00<?, ?it/s]

In [37]:
test_dataset = raw_datasets["test"]
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

In [38]:
pip install --upgrade transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 32.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1
Note: you may need to restart the kernel to use updated packages.


In [39]:
predictions, labels, metrics = trainer.predict(tokenized_test_dataset)

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall']=metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1']=metrics[key]

In [40]:
print(metrics)

{'test_loss': 0.1425502896308899, 'test_LOC_precision': 0.8003169572107766, 'test_LOC_recall': 0.8224755700325733, 'test_LOC_f1': 0.8112449799196788, 'test_LOC_number': 614, 'test_ORG_precision': 0.6602870813397129, 'test_ORG_recall': 0.7885714285714286, 'test_ORG_f1': 0.7187500000000001, 'test_ORG_number': 525, 'test_PER_precision': 0.84192037470726, 'test_PER_recall': 0.910126582278481, 'test_PER_f1': 0.8746958637469586, 'test_PER_number': 790, 'test_overall_precision': 0.7755681818181818, 'test_overall_recall': 0.8491446345256609, 'test_overall_f1': 0.8106904231625834, 'test_overall_accuracy': 0.956718443673654, 'test_runtime': 19.0824, 'test_samples_per_second': 45.434, 'test_steps_per_second': 1.467}


In [41]:
macro_f1 = metrics['test_LOC_f1'] + metrics['test_ORG_f1'] + metrics['test_PER_f1']
print("macro f1 is", macro_f1/3)

macro f1 is 0.8015636145555458


In [43]:
# Import all the necessary classes and initialize the tokenizer and model.
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/tokenizer_indicNER")

model = AutoModelForTokenClassification.from_pretrained("/kaggle/working/my_indicNER")

In [44]:
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file.readlines()]

ground_truth_file = "/kaggle/input/govind-1/govind_1.txt"
corpus = read_file(ground_truth_file)

def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)
    
    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]
    
    predicted_labels = []
    
    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]
    
    return predicted_labels

for sentence in corpus:
    temp = get_ner(sentence)
    print(temp)

[('20/08/2014', 'O'), ('मुख्य', 'O'), ('अभियंता,', 'O'), ('धसान', 'O'), ('केन', 'O'), ('कछार', 'O'), ('सागर', 'O'), ('के', 'O'), ('अन्तर्गत', 'B-LOC'), ('गुण', 'I-ORG'), ('नियंत्रण', 'I-ORG'), ('के', 'I-LOC'), ('कार्यों', 'O'), ('को', 'O'), ('गति', 'O'), ('देने', 'O'), ('हेतु', 'O'), ('मुख्यालय', 'O'), ('परिवर्तन।', 'O')]
[('जानकारी', 'O'), ('है', 'O'), ('कि', 'O'), ('प्रशांत', 'B-PER'), ('किशोर', 'I-PER'), ('ने', 'O'), ('इस', 'O'), ('काम', 'O'), ('के', 'O'), ('लिए', 'O'), ('राहुल', 'B-PER'), ('गांधी', 'I-PER'), ('से', 'O'), ('कोई', 'O'), ('भी', 'O'), ('वेतन', 'O'), ('नहीं', 'O'), ('ले', 'O'), ('रहे', 'O'), ('हैं.', 'O')]
[('यह', 'O'), ('सभी', 'O'), ('वनवासी', 'O'), ('बंधु', 'O'), ('हमारे', 'O'), ('लिए', 'O'), ('नारायण', 'B-PER'), ('का', 'O'), ('ही', 'O'), ('रूप', 'O'), ('हैं।', 'O')]
[('उन्होंने', 'O'), ('मजदूरों', 'O'), ('के', 'O'), ('लिए', 'O'), ('जो', 'O'), ('संबल', 'O'), ('योजना', 'O'), ('बनी', 'O'), ('थी,', 'O'), ('वो', 'O'), ('बंद', 'O'), ('कर', 'O'), ('दी।', 'O')]
[('वहीं', 'O'